# Image Super-Resolution Using Deep Convolutional Networks

单幅图像超分辨率（SR），旨在从一个单一的低分辨率图像恢复高分辨率的图像，是计算机视觉中的经典问题。  
该文章提出了一个全卷积神经网络的方法，直接学习低分辨率图像和高分辨率图像之间的端到端映射，几乎没有预处理/后处理。

## 摘要  

1 对原始分辨率的图像进行下采样和上采样，构建对应的低分辨率图像，作为网络的训练数据，原始分辨率图像作为label  
2 在原始图像上，有重叠的截取大小为33×33的的图像块，作为网络输入x，对应21×21大小的图像块作为label  
3 以网络的输出和label 之间的均方误差作为网络的损失函数，优化网络参数  
4 有人说用训练数据以14为步长，截取图像块，测试数据以21位步长截取图像块，可以缓解block artifacts

### 1 训练数据准备  

1 根据91幅图像，以14位步长，截取大小为33×33的有重叠的图像块，共24800个输入图像，对应的label是24800个21×21大小的高分辨率图像块  
2 输入数据x:将原始图像下采样然后上采样3倍，这样构造低分辨率输入图像，然后有重叠的截取图像块，作为输入  
3 输入数据对应的高分辨率图像label：在原始图像上，有重叠的截取大小为21×21的图像块，作为低分辨率图像对应的高分辨率图像，作为label  
4 网络结构卷积核的大小和个数分别为9x9x64, 1x1x32 和5x5x1的三层卷积网络  
5 33×33大小的图像经过9x9, 1x1和5x5的卷积之后，是21×21大小，可以与label对应起来计算loss

In [ ]:
def preprocess(path, scale=3):
  """
  输入图像作为label，经过采样之后的图像作为低分辨率输入图像
  Preprocess single image file 
    (1) Read original image as YCbCr format (and grayscale as default)
    (2) Normalize
    (3) Apply image file with bicubic interpolation

  Args:
    path: file path of desired file
    input_: image applied bicubic interpolation (low-resolution)
    label_: image with original resolution (high-resolution)
  """
  image = imread(path, is_grayscale=True)
  label_ = modcrop(image, scale)

  # Must be normalized
  image = image / 255.
  label_ = label_ / 255.

  input_ = scipy.ndimage.interpolation.zoom(label_, (1./scale), prefilter=False)
  input_ = scipy.ndimage.interpolation.zoom(input_, (scale/1.), prefilter=False)

  return input_, label_

"""
config.stride : 14
config.image_size : 33
config.label_size : 21
padding : 12/2 = 6
构建网络输入x 和 对应的标签 label
"""
for x in range(0, h-config.image_size+1, config.stride):
    for y in range(0, w-config.image_size+1, config.stride):
      sub_input = input_[x:x+config.image_size, y:y+config.image_size] # [33 x 33]
      sub_label = label_[x+padding:x+padding+config.label_size, y+padding:y+padding+config.label_size] # [21 x 21]

      # Make channel value
      sub_input = sub_input.reshape([config.image_size, config.image_size, 1])  
      sub_label = sub_label.reshape([config.label_size, config.label_size, 1])

      sub_input_sequence.append(sub_input)
      sub_label_sequence.append(sub_label)

### 2 构建网络模型  

1 网络结构十分简单，用了三个卷积层，三个卷积层使用的卷积核的大小和个数分别为9x9x64, 1x1x32 和5x5x1  
2 网络的损失函数也比较简单，是网络输出与label之间的mse

In [ ]:
def build_model(self):
    self.images = tf.placeholder(tf.float32, [None, self.image_size, self.image_size, self.c_dim], name='images')
    self.labels = tf.placeholder(tf.float32, [None, self.label_size, self.label_size, self.c_dim], name='labels')
    
    self.weights = {
      'w1': tf.Variable(tf.random_normal([9, 9, 1, 64], stddev=1e-3), name='w1'),
      'w2': tf.Variable(tf.random_normal([1, 1, 64, 32], stddev=1e-3), name='w2'),
      'w3': tf.Variable(tf.random_normal([5, 5, 32, 1], stddev=1e-3), name='w3')
    }
    self.biases = {
      'b1': tf.Variable(tf.zeros([64]), name='b1'),
      'b2': tf.Variable(tf.zeros([32]), name='b2'),
      'b3': tf.Variable(tf.zeros([1]), name='b3')
    }

    self.pred = self.model()

    # Loss function (MSE)
    self.loss = tf.reduce_mean(tf.square(self.labels - self.pred))
    
def model(self):
    conv1 = tf.nn.relu(tf.nn.conv2d(self.images, self.weights['w1'], strides=[1,1,1,1], padding='VALID') + self.biases['b1'])
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1, self.weights['w2'], strides=[1,1,1,1], padding='VALID') + self.biases['b2'])
    conv3 = tf.nn.conv2d(conv2, self.weights['w3'], strides=[1,1,1,1], padding='VALID') + self.biases['b3']
    return conv3

self.train_op = tf.train.GradientDescentOptimizer(config.learning_rate).minimize(self.loss)

### 3 测试数据准备与输出  

1 根据测试图片截取大小为33×33的有重叠的图像块  
2 送入网络，前向运算，输出结果  
3 merge输出的图像块，拼成测试图像对应的高分辨率图像

In [ ]:
"""
构建测试数据图像块
"""
# Numbers of sub-images in height and width of image are needed to compute merge operation.
nx = ny = 0 
for x in range(0, h-config.image_size+1, config.stride):
  nx += 1; ny = 0
  for y in range(0, w-config.image_size+1, config.stride):
    ny += 1
    sub_input = input_[x:x+config.image_size, y:y+config.image_size] # [33 x 33]
    sub_label = label_[x+padding:x+padding+config.label_size, y+padding:y+padding+config.label_size] # [21 x 21]

    sub_input = sub_input.reshape([config.image_size, config.image_size, 1])  
    sub_label = sub_label.reshape([config.label_size, config.label_size, 1])

    sub_input_sequence.append(sub_input)
    sub_label_sequence.append(sub_label)

"""
前向运行网络
"""
result = self.pred.eval({self.images: train_data, self.labels: train_label})

"""
输出结果merge，存图
"""
result = merge(result, [nx, ny])
result = result.squeeze()
image_path = os.path.join(os.getcwd(), config.sample_dir)
image_path = os.path.join(image_path, "test_image.png")
imsave(result, image_path)

def merge(images, size):
  h, w = images.shape[1], images.shape[2]
  img = np.zeros((h*size[0], w*size[1], 1))
  for idx, image in enumerate(images):
    i = idx % size[1]
    j = idx // size[1]
    img[j*h:j*h+h, i*w:i*w+w, :] = image

  return img